All functions should be run once before to get them to compile w/ numba!!! 
Specific iterarion numbers should be taken from the full numba run so that it's accurate in relation to the others!

In [1]:
import timeit

from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from astropy import units as u
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()
timing_dict_list = {}
timing_dict_list_percall = {}
timing_dict_avg = {}
per_call_dict = {}

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
[py.warnings         ][WARNING]  /ho

In [2]:
"""This is a dictionary of the amount of runs of each function 
based off of 1 run of trace_packet with the above r_packet"""
function_run_dict = {}
function_run_dict["trace_packet"] = 1
function_run_dict["calculate_distance_boundary"] = 1
function_run_dict["calculate_distance_electron"] = 12
function_run_dict["get_doppler_factor"] = 1
function_run_dict["get_doppler_factor_partial_relativity"] = 1
function_run_dict["calculate_distance_line"] = 12
function_run_dict["calculate_tau_electron"] = 12
function_run_dict["update_line_estimators"] = 11
#function_run_dict["test_for_close_line"] = 11

#This can be increased to scale the amount of times the stmt is called
scalar_loop = 20

#This can be increased to increase the number of calls proportionally
scalar_number = 10000
for item in function_run_dict:
    function_run_dict[item] *= scalar_number

In [3]:
import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket, trace_packet)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)

In [4]:
r_packet0 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model0 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma0 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator0 = np.zeros([20])
nu_bar_estimator0 = np.zeros([20])
j_blue_estimator0 = np.zeros([29224, 20])
Edotlu_estimator0 = np.zeros([29224, 20])
estimator0 = Estimators(j_estimator0, nu_bar_estimator0, j_blue_estimator0, Edotlu_estimator0)
r_packet0.initialize_line_id(numba_plasma0, numba_model0)
print(r_packet0.next_line_id)
trace_packet(r_packet0, numba_model0, numba_plasma0, estimator0)
print(r_packet0.next_line_id)

9427
9438


In [5]:
#%%timeit

#trace_packet(r_packet1, numba_model1, numba_plasma1, estimator1)

In [6]:
trace_packet_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket, trace_packet)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator = np.zeros([20])
nu_bar_estimator = np.zeros([20])
j_blue_estimator = np.zeros([29224, 20])
Edotlu_estimator = np.zeros([29224, 20])
estimator = Estimators(j_estimator, nu_bar_estimator, j_blue_estimator, Edotlu_estimator)
r_packet.initialize_line_id(numba_plasma, numba_model)
#This is to have numba compile the function

r_packet0 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model0 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma0 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator0 = np.zeros([20])
nu_bar_estimator0 = np.zeros([20])
j_blue_estimator0 = np.zeros([29224, 20])
Edotlu_estimator0 = np.zeros([29224, 20])
estimator0 = Estimators(j_estimator0, nu_bar_estimator0, j_blue_estimator0, Edotlu_estimator0)
r_packet0.initialize_line_id(numba_plasma0, numba_model0)

trace_packet(r_packet, numba_model, numba_plasma, estimator)
#trace_packet(r_packet0, numba_model0, numba_plasma0, estimator0)


"""
time_sum = 0
timing_dict_list["trace_packet"] = []
timing_dict_list_percall["trace_packet"] = []
for i in range(scalar_loop):
    trace_packet_time = timeit.timeit(setup = trace_packet_setup, stmt='trace_packet(r_packet, numba_model, numba_plasma, estimator)', number = function_run_dict["trace_packet"])
    time_sum += trace_packet_time
    timing_dict_list["trace_packet"].append(trace_packet_time)
    timing_dict_list_percall["trace_packet"].append(trace_packet_time/function_run_dict["trace_packet"])

timing_dict_avg["trace_packet"] = time_sum/scalar_loop
per_call_dict["trace_packet"] = time_sum/(scalar_loop*function_run_dict["trace_packet"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
[py.warnings         ][WARNING]  /ho

[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubuserco

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 

In [7]:
timing_dict_avg["trace_packet"]

0.04224887825002952

In [8]:
r_packet1 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model1 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma1 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator1 = np.zeros([20])
nu_bar_estimator1 = np.zeros([20])
j_blue_estimator1 = np.zeros([29224, 20])
Edotlu_estimator1 = np.zeros([29224, 20])
estimator1 = Estimators(j_estimator1, nu_bar_estimator1, j_blue_estimator1, Edotlu_estimator1)
r_packet1.initialize_line_id(numba_plasma1, numba_model1)
print(r_packet1.next_line_id)
r_inner1 = numba_model1.r_inner[r_packet1.current_shell_id]
r_outer1 = numba_model1.r_outer[r_packet1.current_shell_id]

from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_boundary
calculate_distance_boundary(r_packet1.r, r_packet1.mu, r_inner1, r_outer1)

9427


(54862260430981.75, 1)

In [9]:
calculate_distance_boundary_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize)
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_boundary

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet.initialize_line_id(numba_plasma, numba_model)

r_inner = numba_model.r_inner[r_packet.current_shell_id]
r_outer = numba_model.r_outer[r_packet.current_shell_id]
#This is to have numba compile the function
r_packet1 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model1 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma1 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet1.initialize_line_id(numba_plasma1, numba_model1)
r_inner1 = numba_model1.r_inner[r_packet1.current_shell_id]
r_outer1 = numba_model1.r_outer[r_packet1.current_shell_id]

#calculate_distance_boundary(r_packet1.r, r_packet1.mu, r_inner1, r_outer1)
"""
time_sum = 0
timing_dict_list["calculate_distance_boundary"] = []
timing_dict_list_percall["calculate_distance_boundary"] = []
for i in range(scalar_loop):
    calculate_distance_boundary_time = timeit.timeit(setup = calculate_distance_boundary_setup, stmt='calculate_distance_boundary(r_packet.r, r_packet.mu, r_inner, r_outer)', number = function_run_dict["calculate_distance_boundary"])
    time_sum += calculate_distance_boundary_time
    timing_dict_list["calculate_distance_boundary"].append(calculate_distance_boundary_time)
    timing_dict_list_percall["calculate_distance_boundary"].append(calculate_distance_boundary_time/function_run_dict["calculate_distance_boundary"])

timing_dict_avg["calculate_distance_boundary"] = time_sum/scalar_loop
per_call_dict["calculate_distance_boundary"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_boundary"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.01 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [10]:
timing_dict_avg["calculate_distance_boundary"]

0.0076947849501266315

In [11]:
r_packet2 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model2 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma2 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator2 = np.zeros([20])
nu_bar_estimator2 = np.zeros([20])
j_blue_estimator2 = np.zeros([29224, 20])
Edotlu_estimator2 = np.zeros([29224, 20])
estimator2 = Estimators(j_estimator2, nu_bar_estimator2, j_blue_estimator2, Edotlu_estimator2)
r_packet2.initialize_line_id(numba_plasma2, numba_model2)
print(r_packet2.next_line_id)


from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_electron
cur_electron_density = numba_plasma2.electron_density[r_packet2.current_shell_id]
tau_event = -np.log(np.random.random())
calculate_distance_electron(cur_electron_density, tau_event)

9427


255390632251395.88

In [12]:
calculate_distance_electron_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaPlasma, 
                                                                numba_plasma_initialize)
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_electron                                                                

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)

from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_electron
cur_electron_density = numba_plasma.electron_density[r_packet.current_shell_id]
tau_event = -np.log(np.random.random())
#This is to have numba compile the function
r_packet2 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_plasma2 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)

cur_electron_density1 = numba_plasma2.electron_density[r_packet2.current_shell_id]
tau_event1 = -np.log(np.random.random())

calculate_distance_electron(cur_electron_density, tau_event)
#calculate_distance_electron(cur_electron_density1, tau_event1)
"""
time_sum = 0
timing_dict_list["calculate_distance_electron"] = []
timing_dict_list_percall["calculate_distance_electron"] = []
for i in range(scalar_loop):
    calculate_distance_electron_time = timeit.timeit(setup = calculate_distance_electron_setup, stmt='calculate_distance_electron(cur_electron_density, tau_event)', number = function_run_dict["calculate_distance_electron"])
    time_sum += calculate_distance_electron_time
    timing_dict_list["calculate_distance_electron"].append(calculate_distance_electron_time)
    timing_dict_list_percall["calculate_distance_electron"].append(calculate_distance_electron_time/function_run_dict["calculate_distance_electron"])
    
timing_dict_avg["calculate_distance_electron"] = time_sum/scalar_loop
per_call_dict["calculate_distance_electron"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_electron"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [13]:
timing_dict_avg["calculate_distance_electron"]

0.05002210649990957

In [14]:
r_packet3 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model3 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma3 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator3 = np.zeros([20])
nu_bar_estimator3 = np.zeros([20])
j_blue_estimator3 = np.zeros([29224, 20])
Edotlu_estimator3 = np.zeros([29224, 20])
estimator3 = Estimators(j_estimator3, nu_bar_estimator3, j_blue_estimator3, Edotlu_estimator3)
r_packet3.initialize_line_id(numba_plasma3, numba_model3)
print(r_packet3.next_line_id)

from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor

get_doppler_factor(r_packet3.r, r_packet3.mu, numba_model3.time_explosion)

9427


0.9663192107283891

In [15]:
get_doppler_factor_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel)
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
#This is to have numba compile the function
r_packet3 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model3 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)

get_doppler_factor(r_packet.r, r_packet.mu, numba_model.time_explosion)
#get_doppler_factor(r_packet3.r, r_packet3.mu, numba_model3.time_explosion)
"""
time_sum = 0
timing_dict_list["get_doppler_factor"] = []
timing_dict_list_percall["get_doppler_factor"] = []
for i in range(scalar_loop):
    get_doppler_factor_time = timeit.timeit(setup = get_doppler_factor_setup, stmt='get_doppler_factor(r_packet.r, r_packet.mu, numba_model.time_explosion)', number = function_run_dict["get_doppler_factor"])    
    time_sum += get_doppler_factor_time
    timing_dict_list["get_doppler_factor"].append(get_doppler_factor_time)
    timing_dict_list_percall["get_doppler_factor"].append(get_doppler_factor_time/function_run_dict["get_doppler_factor"])
    
timing_dict_avg["get_doppler_factor"] = time_sum/scalar_loop
per_call_dict["get_doppler_factor"] = time_sum/(scalar_loop*function_run_dict["get_doppler_factor"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [16]:
timing_dict_avg["get_doppler_factor"]

0.007325669249985367

In [17]:
r_packet4 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model4 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma4 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator4 = np.zeros([20])
nu_bar_estimator4 = np.zeros([20])
j_blue_estimator4 = np.zeros([29224, 20])
Edotlu_estimator4 = np.zeros([29224, 20])
estimator4 = Estimators(j_estimator4, nu_bar_estimator4, j_blue_estimator4, Edotlu_estimator4)
r_packet4.initialize_line_id(numba_plasma4, numba_model4)
print(r_packet4.next_line_id)

from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model4.time_explosion
beta = r_packet4.r * inv_t * inv_c

get_doppler_factor_partial_relativity(r_packet4.mu, beta)

9427


0.9663192107283891

In [18]:
get_doppler_factor_partial_relativity_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel)
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model.time_explosion
beta = r_packet.r * inv_t * inv_c
#This is to have numba compile the function
r_packet4 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model4 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)

inv_c1 = 1 / C_SPEED_OF_LIGHT
inv_t1 = 1 / numba_model4.time_explosion
beta1 = r_packet4.r * inv_t1 * inv_c1
get_doppler_factor_partial_relativity(r_packet.mu, beta)
#get_doppler_factor_partial_relativity(r_packet4.mu, beta1)
"""
time_sum = 0
timing_dict_list["get_doppler_factor_partial_relativity"] = []
timing_dict_list_percall["get_doppler_factor_partial_relativity"] = []
for i in range(scalar_loop):
    get_doppler_factor_partial_relativity_time = timeit.timeit(setup = get_doppler_factor_partial_relativity_setup, stmt='get_doppler_factor_partial_relativity(r_packet.mu, beta)', number = function_run_dict["get_doppler_factor_partial_relativity"])
    time_sum += get_doppler_factor_partial_relativity_time
    timing_dict_list["get_doppler_factor_partial_relativity"].append(get_doppler_factor_partial_relativity_time)
    timing_dict_list_percall["get_doppler_factor_partial_relativity"].append(get_doppler_factor_partial_relativity_time/function_run_dict["get_doppler_factor_partial_relativity"])
    
timing_dict_avg["get_doppler_factor_partial_relativity"] = time_sum/scalar_loop
per_call_dict["get_doppler_factor_partial_relativity"] = time_sum/(scalar_loop*function_run_dict["get_doppler_factor_partial_relativity"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [19]:
timing_dict_avg["get_doppler_factor_partial_relativity"]

0.002267676700193988

In [20]:
r_packet5 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model5 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma5 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
j_estimator5 = np.zeros([20])
nu_bar_estimator5 = np.zeros([20])
j_blue_estimator5 = np.zeros([29224, 20])
Edotlu_estimator5 = np.zeros([29224, 20])
estimator5 = Estimators(j_estimator5, nu_bar_estimator5, j_blue_estimator5, Edotlu_estimator5)
r_packet5.initialize_line_id(numba_plasma5, numba_model5)
print(r_packet5.next_line_id)

from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model5.time_explosion
beta = r_packet5.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet5.mu, beta)
comov_nu = r_packet5.nu * doppler_factor

start_line_id = r_packet5.next_line_id
tau_trace_line_combined = 0.0
last_line_id = len(numba_plasma5.line_list_nu) - 1
counter = 0
is_last_line_list = []
nu_line_list = []
comov_nu_list = []
for cur_line_id in range(start_line_id, len(numba_plasma5.line_list_nu)):

    # Going through the lines
    nu_line = numba_plasma5.line_list_nu[cur_line_id]
    nu_line_last_interaction = numba_plasma5.line_list_nu[cur_line_id - 1]

    # Getting the tau for the next line
    tau_trace_line = numba_plasma5.tau_sobolev[
        cur_line_id, r_packet5.current_shell_id
    ]

    # Adding it to the tau_trace_line_combined
    tau_trace_line_combined += tau_trace_line

    # Calculating the distance until the current photons co-moving nu
    # redshifts to the line frequency
    is_last_line = cur_line_id == last_line_id
            
    calculate_distance_line(
        r_packet5,
        comov_nu,
        is_last_line,
        nu_line,
        numba_model5.time_explosion,
    )
    is_last_line_list.append(is_last_line)
    nu_line_list.append(nu_line)
    comov_nu_list.append(comov_nu)
    counter +=   1
print(counter)    

9427
19797


In [21]:
print(comov_nu_list[0])
print(is_last_line_list[0])
print(nu_line_list[0])

764850115187010.4
False
764820807723899.8


In [22]:
print(function_run_dict["calculate_distance_line"])

120000


In [23]:
calculate_distance_line_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line
r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
                         
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet.initialize_line_id(numba_plasma, numba_model)
#This is to have numba compile the function

r_packet5 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model5 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
                         
numba_plasma5 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet5.initialize_line_id(numba_plasma5, numba_model5)

from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity

inv_c1 = 1 / C_SPEED_OF_LIGHT
inv_t1 = 1 / numba_model5.time_explosion
beta1 = r_packet5.r * inv_t1 * inv_c1

doppler_factor1 = get_doppler_factor_partial_relativity(r_packet5.mu, beta1)
comov_nu1 = r_packet5.nu * doppler_factor1
calculate_distance_line(r_packet,764850115187010.4,False,764820807723899,numba_model.time_explosion)
#For some reason doing the one belowq doesnt load the numba function, maybe bc different vars? idk
#calculate_distance_line(r_packet5, comov_nu1, False,764820807723899.8,numba_model5.time_explosion)   
"""
time_sum = 0
timing_dict_list["calculate_distance_line"] = []
timing_dict_list_percall["calculate_distance_line"] = []
for i in range(scalar_loop):
    calculate_distance_line_time = timeit.timeit(setup = calculate_distance_line_setup, stmt='calculate_distance_line(r_packet,764850115187010.4,False,764820807723899,numba_model.time_explosion)', number = function_run_dict["calculate_distance_line"])
    time_sum += calculate_distance_line_time
    timing_dict_list["calculate_distance_line"].append(calculate_distance_line_time)
    timing_dict_list_percall["calculate_distance_line"].append(calculate_distance_line_time/function_run_dict["calculate_distance_line"])
    
timing_dict_avg["calculate_distance_line"] = time_sum/scalar_loop
per_call_dict["calculate_distance_line"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_line"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubuserco

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 

In [24]:
timing_dict_avg["calculate_distance_line"]

0.0940861731998666

In [25]:
r_packet6 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model6 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma6 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet6.initialize_line_id(numba_plasma6, numba_model6)


from tardis.montecarlo.montecarlo_numba.opacities import calculate_tau_electron
cur_electron_density = numba_plasma6.electron_density[r_packet6.current_shell_id]
distance = calculate_distance_line(r_packet6, 764850115187010.4, False, 764820807723899.8, numba_model6.time_explosion)
calculate_tau_electron(cur_electron_density, distance)

0.002376443153769845

In [26]:
calculate_tau_electron_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize)
from tardis.montecarlo.montecarlo_numba.opacities import calculate_tau_electron
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet.initialize_line_id(numba_plasma, numba_model)

cur_electron_density = numba_plasma.electron_density[r_packet.current_shell_id]
distance_trace = calculate_distance_line(r_packet, 764850115187010.4, False, 764820807723899.8, numba_model.time_explosion)
#This is to have numba compile the function

r_packet6 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model6 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma6 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet6.initialize_line_id(numba_plasma6, numba_model6)

cur_electron_density1 = numba_plasma6.electron_density[r_packet6.current_shell_id]
distance1 = calculate_distance_line(r_packet6, 764850115187010.4, False, 764820807723899.8, numba_model6.time_explosion)
calculate_tau_electron(cur_electron_density, distance_trace)
#calculate_tau_electron(cur_electron_density1, distance1)
"""
time_sum = 0
timing_dict_list["calculate_tau_electron"] = []
timing_dict_list_percall["calculate_tau_electron"] = []
for i in range(scalar_loop):
    calculate_tau_electron_time = timeit.timeit(setup = calculate_tau_electron_setup, stmt='calculate_tau_electron(cur_electron_density, distance_trace)', number = function_run_dict["calculate_tau_electron"])
    time_sum += calculate_tau_electron_time
    timing_dict_list["calculate_tau_electron"].append(calculate_tau_electron_time)
    timing_dict_list_percall["calculate_tau_electron"].append(calculate_tau_electron_time/function_run_dict["calculate_tau_electron"])

timing_dict_avg["calculate_tau_electron"] = time_sum/scalar_loop
per_call_dict["calculate_tau_electron"] = time_sum/(scalar_loop*function_run_dict["calculate_tau_electron"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [27]:
print(timing_dict_avg["calculate_tau_electron"])

0.044636905699917406


In [28]:
r_packet7 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model7 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma7 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet7.initialize_line_id(numba_plasma7, numba_model7)

j_estimator7 = np.zeros([20])
nu_bar_estimator7 = np.zeros([20])
j_blue_estimator7 = np.zeros([29224, 20])
Edotlu_estimator7 = np.zeros([29224, 20])
estimator7 = Estimators(j_estimator7, nu_bar_estimator7, j_blue_estimator7, Edotlu_estimator7)

distance_trace = calculate_distance_line(r_packet7, 764850115187010.4, False, 764820807723899.8, numba_model7.time_explosion)
cur_line_id = r_packet7.next_line_id

from tardis.montecarlo.montecarlo_numba.estimators import update_line_estimators
update_line_estimators(estimator7, r_packet7, cur_line_id, distance_trace, numba_model7.time_explosion)

In [29]:
update_line_estimators_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_low_packets.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize,
                                                                Estimators)
from tardis.montecarlo.montecarlo_numba.estimators import update_line_estimators
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line

r_packet = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet.initialize_line_id(numba_plasma, numba_model)

j_estimator = np.zeros([20])
nu_bar_estimator = np.zeros([20])
j_blue_estimator = np.zeros([29224, 20])
Edotlu_estimator = np.zeros([29224, 20])
estimator = Estimators(j_estimator, nu_bar_estimator, j_blue_estimator, Edotlu_estimator)

distance_trace = calculate_distance_line(r_packet, 764850115187010.4, False, 764820807723899.8, numba_model.time_explosion)
cur_line_id = r_packet.next_line_id
#This is to have numba compile the function

r_packet7 = RPacket(1235520000000000.0, 0.9179315093742044, 791508754762811.8, 2.5871368097044846e-05, 2850180890)
numba_model7 = NumbaModel(np.array([1.235520e+15, 1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15]), 
                         np.array([1.286064e+15, 1.336608e+15, 1.387152e+15, 1.437696e+15, 1.488240e+15, 1.538784e+15, 1.589328e+15, 1.639872e+15, 1.690416e+15, 1.740960e+15, 1.791504e+15, 1.842048e+15, 1.892592e+15, 1.943136e+15, 1.993680e+15, 2.044224e+15, 2.094768e+15, 2.145312e+15, 2.195856e+15, 2.246400e+15]), 
                         1123200.0)
numba_plasma7 = numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)
r_packet7.initialize_line_id(numba_plasma7, numba_model7)

j_estimator7 = np.zeros([20])
nu_bar_estimator7 = np.zeros([20])
j_blue_estimator7 = np.zeros([29224, 20])
Edotlu_estimator7 = np.zeros([29224, 20])
estimator7 = Estimators(j_estimator7, nu_bar_estimator7, j_blue_estimator7, Edotlu_estimator7)

distance_trace1 = calculate_distance_line(r_packet7, 764850115187010.4, False, 764820807723899.8, numba_model7.time_explosion)
cur_line_id1 = r_packet7.next_line_id
#update_line_estimators(estimator, r_packet, cur_line_id, distance_trace, numba_model.time_explosion)
#Same as above
#update_line_estimators(estimator7, r_packet7, cur_line_id1, distance_trace1, numba_model7.time_explosion)
"""
time_sum = 0
timing_dict_list["update_line_estimators"] = []
timing_dict_list_percall["update_line_estimators"] = []
for i in range(scalar_loop):
    update_line_estimators_time = timeit.timeit(setup = update_line_estimators_setup, stmt='update_line_estimators(estimator, r_packet, cur_line_id, distance_trace, numba_model.time_explosion)', number = function_run_dict["update_line_estimators"])
    time_sum += update_line_estimators_time
    timing_dict_list["update_line_estimators"].append(update_line_estimators_time)
    timing_dict_list_percall["update_line_estimators"].append(update_line_estimators_time/function_run_dict["update_line_estimators"])

timing_dict_avg["update_line_estimators"] = time_sum/scalar_loop
per_call_dict["update_line_estimators"] = time_sum/(scalar_loop*function_run_dict["update_line_estimators"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo 

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.02 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.26944e+42 erg / s Luminosity absorbed = 2.36367e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 0.03 s (base.py:381)
[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_p

In [30]:
print(timing_dict_avg["update_line_estimators"])

0.12520619189999707


In [31]:
#So this process is somewhat fundamentally flawed, and to make it better I need to do a bit more digging. 
#As I have static packets the total calls doesn't have clear effect, as some may be bigger or smaller. 
#This sum should be less than 100%, as I am not doing test_for_close_line!

In [32]:
for item in timing_dict_list:
    print("{:<30s} {:<30} {:<30}".format(item[:30], "tottime per iter", "avg time percall"))
    for x, items in enumerate(timing_dict_list[item]):
        print("{:30s} {:<30} {:<30}".format(" ", items, timing_dict_list_percall[item][x]))
    print("{:30s}".format(" ") + "-"*60)
    print("{:30s} {:<30} {:<30}".format("averages for time and percall", timing_dict_avg[item], 
                                        per_call_dict[item]))
    print("\n")

trace_packet                   tottime per iter               avg time percall              
                               0.03746813199904864            3.7468131999048637e-06        
                               0.052129256999251083           5.212925699925108e-06         
                               0.045314644001337              4.5314644001337e-06           
                               0.04407646500112605            4.407646500112605e-06         
                               0.03663706999941496            3.663706999941496e-06         
                               0.03445020800063503            3.445020800063503e-06         
                               0.03632575200026622            3.6325752000266222e-06        
                               0.046042969999689376           4.604296999968938e-06         
                               0.04573805300060485            4.573805300060485e-06         
                               0.042028043999380316           4.202804

In [33]:
#This is w/ scalar_loop = 20, scalar_number = 10000

In [34]:
print("{:<30s} {:<30} {:<30} % of trace_packet".format("function", "avg tottime", "avg time percall"))
sum = 0
for item in timing_dict_avg:
    print("{:<30s} {:<30} {:<30} {}".format(item[:30], timing_dict_avg[item], per_call_dict[item], 
                        (float(timing_dict_avg[item]/float(timing_dict_avg["trace_packet"]))*100)))
    if (item != "trace_packet"):
        sum += timing_dict_avg[item]
print("{:<60}  {:<30s} {:<}".format(" ", "sum of subfunctions tottime", (sum/timing_dict_avg["trace_packet"])*100))

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04224887825002952            4.224887825002952e-06          100.0
calculate_distance_boundary    0.0076947849501266315          7.694784950126632e-07          18.212992318017946
calculate_distance_electron    0.05002210649990957            4.168508874992464e-07          118.39866186240013
get_doppler_factor             0.007325669249985367           7.325669249985367e-07          17.339322494272967
get_doppler_factor_partial_rel 0.002267676700193988           2.267676700193988e-07          5.3674246373450245
calculate_distance_line        0.0940861731998666             7.840514433322218e-07          222.69507995706576
calculate_tau_electron         0.044636905699917406           3.719742141659784e-07          105.65228604592885
update_line_estimators         0.12520619189999707            1.1382381081817914e-06         296.3538846144621
       

In [35]:
print("using the same function as stmt, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.019932289549979033           1.993228954997903e-06          100.0
calculate_distance_boundary    0.0033557110500055387          3.3557110500055387e-07         16.83555239146658
calculate_distance_electron    0.026080221449979034           2.173351787498253e-07          130.84408283646707
get_doppler_factor             0.0029159962000107954          2.9159962000107953e-07         14.62950953375982
get_doppler_factor_partial_rel 0.0017899713500355574          1.7899713500355575e-07         8.980259621190584
calculate_distance_line        0.06775552275001928            5.646293562501606e-07          339.9284491634858
calculate_tau_electron         0.021951431649972618           1.8292859708310516e-07         110.13000586275201
update_line_estimators         0.08563682624999273            7.785166022726612e-07          429.63868267748904
                                                              sum of subfunctions tottime    1050.986542086611
""")

using the same function as stmt, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.019932289549979033           1.993228954997903e-06          100.0
calculate_distance_boundary    0.0033557110500055387          3.3557110500055387e-07         16.83555239146658
calculate_distance_electron    0.026080221449979034           2.173351787498253e-07          130.84408283646707
get_doppler_factor             0.0029159962000107954          2.9159962000107953e-07         14.62950953375982
get_doppler_factor_partial_rel 0.0017899713500355574          1.7899713500355575e-07         8.980259621190584
calculate_distance_line        0.06775552275001928            5.646293562501606e-07          339.9284491634858
calculate_tau_electron         0.021951431649972618           1.8292859708310516e-07         110.13000586275201
update_line_estimators         0.085636826249992

In [36]:
print("using similar function to try and primte it, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.02017318594998869            2.017318594998869e-06          100.0
calculate_distance_boundary    0.0035736684500420777          3.573668450042078e-07          17.714943286110348
calculate_distance_electron    0.02552785034996532            2.12732086249711e-07           126.54347416045917
get_doppler_factor             0.0028511989999969955          2.851198999996996e-07          14.133607884572113
get_doppler_factor_partial_rel 0.0018525005499554937          1.8525005499554935e-07         9.18298455458659
calculate_distance_line        0.07088738119999789            5.907281766666491e-07          351.3940801206843
calculate_tau_electron         0.023377500600020085           1.9481250500016738e-07         115.88402871998109
update_line_estimators         0.08447746764995827            7.679769786359843e-07          418.76116077741113
                                                              sum of subfunctions tottime    1053.6142795038047
""")

using similar function to try and primte it, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.02017318594998869            2.017318594998869e-06          100.0
calculate_distance_boundary    0.0035736684500420777          3.573668450042078e-07          17.714943286110348
calculate_distance_electron    0.02552785034996532            2.12732086249711e-07           126.54347416045917
get_doppler_factor             0.0028511989999969955          2.851198999996996e-07          14.133607884572113
get_doppler_factor_partial_rel 0.0018525005499554937          1.8525005499554935e-07         9.18298455458659
calculate_distance_line        0.07088738119999789            5.907281766666491e-07          351.3940801206843
calculate_tau_electron         0.023377500600020085           1.9481250500016738e-07         115.88402871998109
update_line_estimators         0.08

In [37]:
print("no 'priming function' in setup, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.02094911165003168            2.094911165003168e-06          100.0
calculate_distance_boundary    0.003327815850013849           3.3278158500138487e-07         15.885236116963538
calculate_distance_electron    0.02591074014995911            2.1592283458299258e-07         123.6841952194184
get_doppler_factor             0.002841918399985843           2.8419183999858435e-07         13.565818195357918
get_doppler_factor_partial_rel 0.0017482770500237166          1.7482770500237166e-07         8.345351722926509
calculate_distance_line        0.07204161454999394            6.003467879166162e-07          343.8886371579627
calculate_tau_electron         0.021481245900008616           1.790103825000718e-07          102.54012799619659
update_line_estimators         0.0845031700500158             7.682106368183255e-07          403.37352467109514
                                                              sum of subfunctions tottime    1011.2828910799207
""")

no 'priming function' in setup, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.02094911165003168            2.094911165003168e-06          100.0
calculate_distance_boundary    0.003327815850013849           3.3278158500138487e-07         15.885236116963538
calculate_distance_electron    0.02591074014995911            2.1592283458299258e-07         123.6841952194184
get_doppler_factor             0.002841918399985843           2.8419183999858435e-07         13.565818195357918
get_doppler_factor_partial_rel 0.0017482770500237166          1.7482770500237166e-07         8.345351722926509
calculate_distance_line        0.07204161454999394            6.003467879166162e-07          343.8886371579627
calculate_tau_electron         0.021481245900008616           1.790103825000718e-07          102.54012799619659
update_line_estimators         0.084503170050015